#### Importing mysql connector package

In [ ]:
import mysql.connector as mysql

## Python-Mysql Without Using Class

#### Creating connection

In [ ]:
try:
    
    mydb = mysql.connect(host='localhost', user='root', password='root')
    
except Exception as e:
    
    print('Failed to create connection with mysql server, reason {0}'.format(e))

In [ ]:
print(mydb)

#### Creating cursor to interact with MySQL server

In [ ]:
try:
    
    cursor = mydb.cursor(buffered=True)

except Exception as e:
    
    print('Error occur while creating cursor object, reason {0}'.format(e))

##### Reason why to use buffered?
> https://stackoverflow.com/questions/54518722/mysql-connector-could-not-process-parameters

#### Creating database using python code

In [ ]:
try:
    
    query  = "create database testing"
    cursor.execute(query)
    
except Exception as e:
    
    print('Error occure while executing query: {0}, reason {1}'.format(query, e))

Error occure while executing query: create database testing, reason 1007 (HY000): Can't create database 'testing'; database exists


In [ ]:
try:
    pass
except Exception as e:
    pass

#### Showing all the databases

In [ ]:
try:
    
    query   = 'show databases'
    cursor.execute(query)
    results = cursor.fetchall() # return all results in list
    
    for dbname in results:
        print(dbname)
    
except Exception as e:
    
    print('Error occuring while executing query: {0}, reason {1}'.format(query, e))

('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('testing',)


#### Connecting to a database

In [ ]:
try:
    
    dbname    = 'testing'
    testingdb = mysql.connect(host='localhost', user='root', password='root', database=dbname)
    print(testingdb)
    
except Exception as e:
    
    print('Error occur while connecting to database, reeason {0}'.format(e))

## Python-MySQL Using Class

In [ ]:
class DbStreamer:

    def __init__(self, host, user, password, database):
        self.conn = mysql.connect(host=host, user=user, passwd=password, db=database)
        return

    def get_connection(self):
        return self.conn

    def close_connection(self):
        self.conn.commit()
        self.conn.close()
        return

    def insert_user_details_sanitized(self, name, rollnum, userid):
        sql = "INSERT INTO STUDENT (name, rollnum, userid) VALUES (%s, %s, %s)"
        val = (name, rollnum, userid)

        _cursor = self.conn.cursor()
        _cursor.execute(sql, val)
        return

    def insert_user_details_unsanitized(self, name, rollnum, userid):
        sql = "INSERT INTO STUDENT (name, rollnum, userid) VALUES ('%s', '%s', '%s')" % (name, rollnum, userid)

        _cursor = self.conn.cursor()
        _cursor.execute(sql)
        return
    
    def get_tables(self):
        sql = "SHOW TABLES;"
        
        _cursor = self.conn.cursor()
        _cursor.execute(sql)
        data = _cursor.fetchall()
        
        print(data)
        return

    def get_all_users(self):
        try:
            sql = "SELECT * FROM STUDENT"

            _cursor = self.conn.cursor()
            _cursor.execute(sql)

            row_headers = [x[0] for x in _cursor.description]
            data = _cursor.fetchall()

            json_data = []
            for result in data:
                json_data.append(dict(zip(row_headers, result)))
            print(json_data, end="\n\n")
        except Exception as ex:
            print(ex)
        return

### Initiate the connection

In [ ]:
db_streamer = DbStreamer("localhost", user='root', password='root', database="testing")

### Get tables and all entries in user table

In [ ]:
db_streamer.get_tables()
db_streamer.get_all_users()

[]
1146 (42S02): Table 'testing.STUDENT' doesn't exist


### Sanitized input addition

In [ ]:
db_streamer.insert_user_details_sanitized(name='eren yeager', rollnum='25', userid="Alex C'); DROP TABLE STUDENT;")
db_streamer.get_tables()
db_streamer.get_all_users()
# To show all the updates in the database and table, you have to run commit function
db_streamer.close_connection() # After this, all the updates have been made to database

[('STUDENT',)]
[{'name': 'eren yeager', 'rollnum': '25', 'userid': "Alex C'); DROP TABLE STUDENT;"}]



### Unsanitized input

In [ ]:
db_streamer = DbStreamer("localhost", user='root', password='root', database="testing")

In [ ]:
try:
    # Carefule, after executing this query , connection to mysql server might be closed
    # But query will show its effect, and table will be dropped
    db_streamer.insert_user_details_unsanitized(name='light yagami', rollnum='21', userid="Alex B'); DROP TABLE STUDENT;")
    db_streamer.get_all_users()
    db_streamer.get_tables()
except Exception as e:
    print('Error occur while executing query, reason {}'.format(e))

#### References : https://github.com/shruti-singh/CS432Database/blob/master/DatabaseConnector.ipynb